In [1]:
import psycopg2
import numpy as np
import pandas as pd

# 경고 처리
import warnings
warnings.filterwarnings('ignore')

# float 자리수 설정
pd.options.display.float_format = '{:.0f}'.format

# DB Connect
conn = psycopg2.connect(
    host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port=5439,
    dbname='dev',
    user='awsuser',
    password='cremaoAdmin1234qwer!!'
)
cursor = conn.cursor()

### 1. 연별 합계 sum

In [52]:
# 유동인구
# 2020
query = 'select sum(f_tot) from m1.kt_bldg_xy_info where base_ym between 202001 and 202012' # 쿼리문
cursor.execute(query) # 실행
f_tot_2020 = cursor.fetchall() # 출력
f_tot_2020 = f_tot_2020[0][0] # 값 꺼내기

# 2021
query = 'select sum(f_tot) from m1.kt_bldg_xy_info where base_ym between 202101 and 202112'
cursor.execute(query)
f_tot_2021 = cursor.fetchall()
f_tot_2021 = f_tot_2021[0][0]

# 2022
query = 'select sum(f_tot) from m1.kt_bldg_xy_info where base_ym between 202201 and 202212'
cursor.execute(query)
f_tot_2022 = cursor.fetchall()
f_tot_2022 = f_tot_2022[0][0]

In [53]:
# 거주인구
# 2020
query = 'select sum(pop) from m1.kt_bldg_xy_info where base_ym between 202001 and 202012'
cursor.execute(query)
pop_2020 = cursor.fetchall()
pop_2020 = pop_2020[0][0]

# 2021
query = 'select sum(pop) from m1.kt_bldg_xy_info where base_ym between 202101 and 202112'
cursor.execute(query)
pop_2021 = cursor.fetchall()
pop_2021 = pop_2021[0][0]


# 2022
query = 'select sum(pop) from m1.kt_bldg_xy_info where base_ym between 202201 and 202212'
cursor.execute(query)
pop_2022 = cursor.fetchall()
pop_2022 = pop_2022[0][0]

In [54]:
# 직장인구
# 2020
query = 'select sum(wpop) from m1.kt_bldg_xy_info where base_ym between 202001 and 202012'
cursor.execute(query)
wpop_2020 = cursor.fetchall()
wpop_2020 = wpop_2020[0][0]

# 2021
query = 'select sum(wpop) from m1.kt_bldg_xy_info where base_ym between 202101 and 202112'
cursor.execute(query)
wpop_2021 = cursor.fetchall()
wpop_2021 = wpop_2021[0][0]

# 2022
query = 'select sum(wpop) from m1.kt_bldg_xy_info where base_ym between 202201 and 202212'
cursor.execute(query)
wpop_2022 = cursor.fetchall()
wpop_2022 = wpop_2022[0][0]

### 1. 월별 합계 (sum)

In [2]:
# 월별 합(sum)
def getData(col, df):
    
    year_list  = pd.date_range(start='2020', end='2022', freq='YS').strftime('%Y')
    month_list = pd.date_range(start='2021.01', end='2021.12', freq='MS').strftime('%m')

    for year in year_list:

        for month in month_list:

            query = f'select sum({col}) from m1.kt_bldg_xy_info where base_ym = {year}{month}'
            cursor.execute(query)
            data = cursor.fetchall()
            df_sub = pd.DataFrame([data[0][0]], columns=[year+month]).T
            df = df.append(df_sub)

    return df

# base 데이터프레임 생성
pop   = pd.DataFrame() # 거주인구
wpop  = pd.DataFrame() # 직장인구
f_tot = pd.DataFrame() # 유동인구

# 월별 합 데이터프레임화
pop   = getData('pop',pop)      # 거주인구
wpop  = getData('wpop',wpop)   # 직장인구
f_tot = getData('f_tot', f_tot) # 유동인구

In [3]:
# 데이터 쏠림 정도 파악
f_tot.describe()

,0
count,32
mean,4382727171089
std,367832846661
min,3618135717697
25%,4199154591963
50%,4498697527770
75%,4633358084166
max,4949080181901


### 2. 전년대비 증감율

In [4]:
# 전년 대비 증감율 
def perYearIncrease(col, df):
    
    # 변수 생성
    df[col] = 0
    year_list  = pd.date_range(start='2020', end='2022', freq='YS').strftime('%Y')
    month_list = pd.date_range(start='2021.01', end='2021.12', freq='MS').strftime('%m')

    # 전년 대비 값 비교
    for month in month_list:

        for i in range(len(df)):

            # 2020년 대비 2021년 (120% 이상: 1, 120% 미만: 0)
            if (((f_tot.loc[year_list[1]+month]) / (f_tot.loc[year_list[0]+month]))*100 >= 60).bool == True:
                df[col][i] = 1
            else:
                df[col][i] = 0
                
            # 2021년 대비 2022년
            if ((((f_tot.loc[year_list[2]+month]) / (f_tot.loc[year_list[1]+month]))*100) >= 60).bool == True:
                df[col][i] = 1
            else:
                df[col][i] = 0
    return df

# 데이터프레임화
pop   = perYearIncrease('pop', pop)      # 거주인구
wpop  = perYearIncrease('wpop', wpop)    # 직장인구
f_tot = perYearIncrease('f_tot', f_tot)  # 유동인구

In [227]:
pop

,0,pop
202001,8480241170,0
202002,8655889933,0
202003,8651198655,0
202004,8741215040,0
202005,8877044107,0
...,...,...
202208,8472401816,0
202209,NaN,0
202210,NaN,0
202211,NaN,0


In [ ]:
# NaN 처리
pop   = pop.drop(pop[pop[0].isnull()].index, axis=0)       # 거주인구
wpop  = wpop.drop(wpop[wpop[0].isnull()].index, axis=0)    # 직장인구
f_tot = f_tot.drop(f_tot[f_tot[0].isnull()].index, axis=0) # 유동인구